In [6]:
import numpy as np
import pandas as pd
import re, sys, os, csv
# from many_stop_words import get_stop_words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import emoji

In [7]:
#stop_words = list(get_stop_words('en'))         #About 900 stop words
#nltk_words = list(stopwords.words('english'))   #About 150 stop words
#stop_words.extend(nltk_words)

def word_prob(word): return dictionary[word] / total
def words(text): return re.findall('[a-z]+', text.lower())
dictionary = Counter(words(open('dataset/wordlists/merged.txt').read()))
max_word_length = max(map(len, dictionary))
total = float(sum(dictionary.values()))

def viterbi_segment(text):
    probs, lasts = [1.0], [0]
    for i in range(1, len(text) + 1):
        prob_k, k = max((probs[j] * word_prob(text[j:i]), j)
                        for j in range(max(0, i - max_word_length), i))
        probs.append(prob_k)
        lasts.append(k)
    words = []
    i = len(text)
    while 0 < i:
        words.append(text[lasts[i]:i])
        i = lasts[i]
    words.reverse()
    return words, probs[-1]

def fix_hashtag(text):
    text = text.group().split(":")[0]
    text = text[1:] # remove '#'
    try:
        test = int(text[0])
        text = text[1:]
    except:
        pass
    output = ' '.join(viterbi_segment(text)[0])
    #print(output)
    return output

def remove_emoji(text):
    import re
    emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text) # no emoji

def clean_tweet( tweet):
        tweet = tweet.lower()
        tweet = remove_emoji(tweet)
        tweet = re.sub("(#[A-Za-z0-9]+)", fix_hashtag, tweet)
        
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def remove_stopwords(word_list):
        filtered_tweet=""
        for word in word_list:
            word = word.lower() 
            if word not in stopwords.words("english"):
                filtered_tweet=filtered_tweet + " " + word
        
        return filtered_tweet.lstrip()
    
def vectorise_label(label):
    if label == "happy": return 1
    elif label == "sad": return 2
    elif label == "surprise": return 3
    elif label == "worry": return 4
    elif label == "disgust": return 5
    elif label == "annoy": return 6
    elif label == "joke": return 7
    elif label == "intrigue": return 8
    elif label == "awesome": return 9
    elif label == "charm": return 104
    
    
    elif label == "empty":return 105
    elif label == "neutral":return 105
    elif label == "happiness":return 1
    elif label == "enthusiasm":return 1
    elif label == "sadness":return 2
    elif label == "hate":return 5
    elif label == "anger":return 6
    
    elif label == "fun":return 100 # happy
    elif label == "boredom":return 101 # neutral
    elif label == "relief":return 102 # happy
    elif label == "love":return 103
    

    
    

In [8]:
data_train = pd.read_csv('dataset/data/text_emotion.csv', sep=',')
print("Dataset shape:",data_train.shape)
print(data_train.sentiment[0],":",data_train.content[0])

Dataset shape: (284436, 4)
charm : @spookymoonkitty I just picked one up. I’m in love with him 😍


In [9]:
dataWriter = csv.writer(open('data.csv', 'w'), delimiter=',',lineterminator="\n")

# total = 40000
total = len(data_train)
from tqdm import tqdm
for i in tqdm(range(total)):
    #print("Progress: ",round(i/total*100,2),"   ",end="\r")
    tweet= clean_tweet(data_train.content[i])
    #tweet = remove_stopwords(tweet.split())
    dataWriter.writerow([tweet, str(vectorise_label(data_train.sentiment[i]))])
    #sys.stdout.write("\033[F")
    
print("Progress: ",100,"\nComplete!")

100%|██████████| 284436/284436 [00:36<00:00, 7827.78it/s]

Progress:  100 
Complete!


In [10]:
count = 0
with open('data.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        count+=1

In [11]:
count

284428